### Matching

In [1]:
import networkx as nx
from scipy.spatial import distance
from tqdm import tqdm
import pandas as pd
import numpy as np

import concurrent.futures

pd.set_option('display.max_columns', None)

In [2]:
# Cols that we want to Match
cols_m = [
    'log Area Comercial',
    'log Area Residencial - V',
    'log Area Residencial - H',
    'log Area Terreno',
    'CA',
    'log VALOR DO M2 DE CONSTRUCAO'
]
N_MATCHES = 3

In [3]:
# Read the data
df_results = pd.read_parquet('data/df_results_700.parquet')


df_results['log VALOR DO M2 DE CONSTRUCAO'] = np.log(df_results['VALOR DO M2 DE CONSTRUCAO'] + 10)
df_results['log VALOR DO M2 DO TERRENO'] = np.log(df_results['VALOR DO M2 DO TERRENO'] + 10)

df_results.sample(10)

,sq,geometry,Nome,Linha,elegivel,T,Inauguração,ANO DO EXERCICIO,AREA OCUPADA,AREA DO TERRENO,AREA CONSTRUIDA,Area Comercial,Area Residencial - H,Area Residencial - V,Area Terreno,Residencial - H,Residencial - V,Comercial,Terreno,QUANTIDADE DE PAVIMENTOS,VALOR DO M2 DE CONSTRUCAO,VALOR DO M2 DO TERRENO,Valor,ANO DE INICIO DA VIDA DO CONTRIBUINTE,CA,TO,ano,delta,per Area Comercial,per Area Residencial - V,per Area Residencial - H,diff AREA CONSTRUIDA,diff Area Comercial,diff Area Residencial - H,diff Area Residencial - V,sum AREA CONSTRUIDA,sum Area Comercial,sum Area Residencial - H,sum Area Residencial - V,log AREA CONSTRUIDA,log Area Comercial,log Area Residencial - H,log Area Residencial - V,log Area Terreno,log VALOR DO M2 DE CONSTRUCAO,log VALOR DO M2 DO TERRENO
271092,123201,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00M\x00\x0...,Borba Gato,5,1,0,2017-06-09,2019,2496,18245,4218,0,4217,0,6118,0.750000,0.0,0.000000,0.250000,2.149123,1511.375000,1373.875000,8.873618,1966.111190,0.231187,0.136805,2017,2,0.000000,0.0,0.999763,0.0,0.0,0.0,0.0,-731.0,0.0,-731.0,0.0,8.347116,0.000000,8.346879,0.0,8.718991,7.327370,7.232643
230453,089393,b'\x01\x03\x00\x00\x00\x01\x00\x00\x007\x00\x0...,Borba Gato,5,1,0,2017-06-09,2001,2300,4512,4045,0,4044,0,0,1.000000,0.0,0.000000,0.000000,1.892707,278.612903,220.741935,1.143917,1970.308776,0.896498,0.509752,2017,-16,0.000000,0.0,0.999753,0.0,0.0,0.0,0.0,-20.0,0.0,-20.0,0.0,8.305237,0.000000,8.304990,0.0,0.000000,5.665086,5.441300
326651,123212,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00=\x00\x0...,Alto da Boa Vista,5,1,0,2017-06-09,2001,1692,5374,3473,0,3472,0,915,0.818182,0.0,0.000000,0.181818,1.924273,400.181818,196.454545,1.739216,1964.872445,0.646260,0.314849,2017,-16,0.000000,0.0,0.999712,0.0,0.0,0.0,0.0,1205.0,0.0,1205.0,0.0,8.152774,0.000000,8.152486,0.0,6.818924,6.016601,5.330080
240493,101082,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00U\x00\x0...,Vila Sônia,4,1,0,2019-12-17,2014,3864,7092,5932,2943,2988,0,0,0.718750,0.0,0.281250,0.000000,1.691841,733.156250,421.250000,4.692734,1969.713082,0.836435,0.544839,2019,-5,0.496123,0.0,0.503709,0.0,0.0,0.0,0.0,1095.0,830.0,265.0,0.0,8.688117,7.987185,8.002360,0.0,0.000000,6.610906,6.066688
91226,155305,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x8d\x00...,Vila União,15,1,1,2018-06-04,2025,10472,18182,15471,4112,11158,0,152,0.925234,0.0,0.046729,0.009346,1.704932,1499.962617,1110.168224,25.041396,1988.581927,0.850896,0.575954,2018,7,0.265788,0.0,0.721220,0.0,0.0,0.0,0.0,2952.0,193.0,2609.0,0.0,9.646723,8.321665,9.319912,0.0,5.023881,7.319840,7.021234
691589,165359,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00)\x00\x0...",Campo Limpo,5,1,0,2002-10-20,2025,438,1091,615,0,614,0,214,0.750000,0.0,0.000000,0.250000,1.559350,1088.625000,669.375000,0.913365,1968.030894,0.563703,0.401467,2002,23,0.000000,0.0,0.998374,0.0,0.0,0.0,0.0,-35.0,0.0,-35.0,0.0,6.421622,0.000000,6.419995,0.0,5.365976,7.001815,6.521173
114778,084053,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00@\x00\x0...",Moema,5,1,0,2018-05-04,2009,5040,12190,8198,0,8197,0,0,1.000000,0.0,0.000000,0.000000,1.895462,818.800000,523.000000,6.760696,1963.226153,0.672518,0.413454,2018,-9,0.000000,0.0,0.999878,0.0,0.0,0.0,0.0,1324.0,0.0,1324.0,0.0,9.011646,0.000000,9.011524,0.0,0.000000,6.719979,6.278521
310127,300103,b'\x01\x03\x00\x00\x00\x01\x00\x00\x002\x00\x0...,Adolfo Pinheiro,5,1,0,2014-12-02,2015,2235,12210,3255,0,3254,0,0,1.000000,0.0,0.000000,0.000000,1.733333,2085.714286,1565.000000,6.863640,1985.828264,0.266585,0.183047,2014,1,0.000000,0.0,0.999693,0.0,0.0,0.0,0.0,703.0,0.0,703.0,0.0,8.087948,0.000000,8.087640,0.0,0.000000,7.647650,7.362011
206742,080134,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00I\x00\x0...",Pinheiros,4,1,0,2011-05-16,1997,3464,6575,4746,395,4350,0,398,0.888889,0.0,0.083333,0.027778,1.606195,237.138889,233.000000,1.197283,1964.168984,0.721825,0.526844,2011,-14,0.083228,0.0,0.916561,-23.0,0.0,-23.0,0.0,-23.0,0.0,-23.0,0.0,8.465057,5.978886,8.377931,0.0,5.986452,5.509950,5.493061
432254,051158,b'\x01\x03\x0

In [4]:
import numpy as np

def matching_algo(treated, control):
    # Create an empty graph
    graph = nx.Graph()

    # Add treated units to the graph as nodes
    for _, treated_row in treated.iterrows():
        graph.add_node(treated_row.sq, bipartite=0)

    # Add control units to the graph as nodes
    for _, control_row in control.iterrows():
        graph.add_node(control_row.sq, bipartite=1)

    # We iterate: for each treated and for each control unitm
    # We calculate the distance in respespect over the macthing columns
    # The distante is the weight edge
    for _, treated_row in treated.iterrows():
        for _, control_row in control.iterrows():
            dist = distance.euclidean(treated_row[cols_m], control_row[cols_m])
            graph.add_edge(treated_row.sq, control_row.sq, weight=dist)
    
    # So we calculate the distance for each pair
    # So now we apply the algorithm
    # It'll return the perfect match which minimize the distance
    matching = nx.bipartite.minimum_weight_full_matching(graph)
    return matching


def finding_matching(df, nome):
    # Select the ids columns + Matching columns
    cols = ['Nome', 'sq', 'T'] + cols_m
    
    # Filter only the data we need
    # Group we want and the prior
    df_f = df_results[
        (df.delta == -3) 
        & (df.Nome == nome)
    ][cols]
    
    # Separate treated and control units    
    treated = df_f[df_f['T'] == 1]
    control = df_f[df_f['T'] == 0]
    
    # Apply Matching algorithm
    matching = assignment_ilp(treated, control, n_matches=N_MATCHES)
    
    matched_controls = list({ctrl for sublist in matching.values() for ctrl in sublist})
    # Return DataFrame
    df_matching = pd.concat([
        control[control.sq.isin(matched_controls)],
        treated
    ])
    return df_matching

In [5]:
import pandas as pd
from scipy.spatial import distance
import pulp

def assignment_ilp(treated: pd.DataFrame,
                   control: pd.DataFrame,
                   n_matches: int = 3):
    """
    Resolve o problema de pareamento 1 para n via Programação Inteira Linear.

    Parâmetros:
    - treated: DataFrame com unidades tratadas e coluna 'sq'.
    - control: DataFrame com unidades de controle e coluna 'sq'.
    - cols_m: lista de colunas a considerar na distância.
    - n_matches: número de controles por tratado (n).

    Retorna:
    - dict: mapeamento de cada tratado para lista de controles.
    """
    # Índices
    T = list(treated['sq'])
    C = list(control['sq'])

    # Construção da matriz de custos c[i,j]
    cost = {}
    for _, t_row in treated.iterrows():
        for _, c_row in control.iterrows():
            cost[(t_row.sq, c_row.sq)] = distance.euclidean(
                t_row[cols_m], c_row[cols_m]
            )

    # Definição do modelo de otimização
    model = pulp.LpProblem('Assignment_1_to_n', pulp.LpMinimize)

    # Variáveis binárias x[i][j]
    x = pulp.LpVariable.dicts('x', (T, C), cat=pulp.LpBinary)

    # Função-objetivo: minimizar soma dos custos
    model += pulp.lpSum(cost[(i, j)] * x[i][j] for i in T for j in C), 'Total_Cost'

    # Restrição: cada tratado pareado com exatamente n_matches controles
    for i in T:
        model += pulp.lpSum(x[i][j] for j in C) == n_matches, f'Restrict_Treated_{i}'

    # Restrição: cada controle pareado com no máximo um tratado
    for j in C:
        model += pulp.lpSum(x[i][j] for i in T) <= 1, f'Restrict_Control_{j}'

    # Resolver o modelo
    solver = pulp.PULP_CBC_CMD(msg=False)
    model.solve(solver)

    # Extrair pareamentos
    matching = {
        i: [j for j in C if pulp.value(x[i][j]) == 1]
        for i in T
    }
    return matching

In [6]:
# Selecting all the Metro Stations Groups to iterate over
aux_list_result = set(df_results.Nome.unique())

# This process takes timing
# To avoid that, we use a multiprocessing.
# That means, instead of iterating one at time, we have 8 workers
with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
    # Apply algorithm
    futures = [executor.submit(finding_matching, df_results, i) for i in aux_list_result]
    # Collecting the resulting and tracking the processing
    results = [future.result() for future in tqdm(concurrent.futures.as_completed(futures))]

44it [03:47,  5.18s/it]


In [7]:
# Concat the results into a unified dataframe
df_results_matching = pd.concat(results)

In [8]:
# Lets see before the matching
pd.DataFrame(df_results[df_results.delta == -3].groupby(['T'])[cols_m].mean()).round(2)

,log Area Comercial,log Area Residencial - V,log Area Residencial - H,log Area Terreno,CA,log VALOR DO M2 DE CONSTRUCAO
T,,,,,,
0,4.67,1.47,7.21,3.83,0.82,6.04
1,6.05,2.94,7.06,2.91,1.07,6.32


In [9]:
pd.DataFrame(df_results[df_results.delta == -3].groupby(['T'])[cols_m].count())

,log Area Comercial,log Area Residencial - V,log Area Residencial - H,log Area Terreno,CA,log VALOR DO M2 DE CONSTRUCAO
T,,,,,,
0,22434,22434,22434,22434,22434,22434
1,3062,3062,3062,3062,3062,3062


In [10]:
# Apply an inner join to select only the matched units
df_results_matching_all = pd.merge(
    df_results,
    df_results_matching[['Nome', 'sq', 'T']],
    on = ['Nome', 'sq', 'T']
)
# Let's compareA after matching
df_results_matching_all[df_results_matching_all.delta == -3].groupby(['T'])[cols_m].mean().round(2)

,log Area Comercial,log Area Residencial - V,log Area Residencial - H,log Area Terreno,CA,log VALOR DO M2 DE CONSTRUCAO
T,,,,,,
0,5.91,2.20,7.29,3.22,0.97,6.30
1,6.05,2.94,7.06,2.91,1.07,6.32


In [11]:
df_results_matching_all[df_results_matching_all.delta == -3].groupby(['T'])[cols_m].count()

,log Area Comercial,log Area Residencial - V,log Area Residencial - H,log Area Terreno,CA,log VALOR DO M2 DE CONSTRUCAO
T,,,,,,
0,8835,8835,8835,8835,8835,8835
1,3062,3062,3062,3062,3062,3062


In [12]:
df_results_matching_all.to_parquet('data/did_matching_700.parquet')